# Surprise 라이브러리를 사용한 추천시스템

## Surprise
### - 파이썬 기반 추천시스템 구축을 위한 전용패키지
### - 사이킷런과 유사한 API와 프레임워크 제공

## 1. Surprise 설치

### pip install scikit-surprise
### 또는
### conda install -C conda-forge scikit-surprise

### 설치시 다음과 같은 오류 발생하는경우
### Microsoft Visual C++ 14.0 , C++를 사용한 데스크톱 개발 설치
### https://visualstudio.microsoft.com/ko/vs/community/


## 2.Surprise 클래스 사용법

### (1)  관련모듈 import

In [3]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

### (2) Surprise 사용법 익히기

#### sample data : Surprise 패키지에서 movielens 사이트에서 제공하는 과거 영화데이터 세트 제공
#### ml-100k (10만개의 평점 데이터 포함)

In [8]:
# 패키지에 포함된 내장 데이터 읽기
data = Dataset.load_builtin('ml-100k')
train_set, test_set = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] 

 y


Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\602-01/.surprise_data/ml-100k


In [9]:
# 경사적 하강법 알고리즘으로 학습
svd = SVD(random_state=0)
svd.fit(train_set)

In [11]:
# 전체 데이터 세트에 대한 예측: test()
predictions = svd.test(test_set)
print('prediction_type : ', type(predictions), ' size:', len(predictions))
print('prdiction 결과 최초 5개 추출 : ')
predictions[:5]

prediction_type :  <class 'list'>  size: 25000
prdiction 결과 최초 5개 추출 : 


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

#### predictions 컬럼 의미
#### uid: 사용자 아이디
#### iid: 영화 아이디
#### r_ui: 실제 평점
#### est: 예측 평점
#### was_impossible 이 'True'이면 추천 예측값을 생성할 수 없는 데이터

In [12]:
# predictions 객체에서 uid, iid, est 속성 추출
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [13]:
# 임의의 사용자로 추천 예측
uid = str(196)
iid = str(302)
pred = svd.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


#### predict()는 개별 사용자와 아이템 정보를 입력하면 예측평점을 est로 반환

#### 추천 예측평점과 실제 평점 사이의 평가

#### rmse() 사용 - Root Mean Sqouard Error(평균 제곱근 오차)

In [14]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

### 로컬 컴퓨터의 데이터를 로딩하여 테스트

In [20]:
import pandas as pd

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
# Surprise 패키지에서 사용시 인덱스와 헤더 모두 제거하여 사용
ratings.to_csv('./ml-latest-small/ratings_noh.csv', index=False, header=False)

In [21]:
# Dataset 로드
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5.0))
data = Dataset.load_from_file('./ml-latest-small/ratings_noh.csv', reader=reader)

#### SVD를 이용한 추천 예측

In [23]:
train_set, test_set = train_test_split(data, test_size=0.25, random_state=0)
# n_factors : 잠재요인 크기
svd = SVD(n_factors=50, random_state=0)

svd.fit(train_set)
predictions = svd.test(test_set)
accuracy.rmse(predictions)


RMSE: 0.8682


0.8681952927143516

### 판다스 데이터프레임에서 데이터 세트 로딩할 경우

In [29]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('./ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
train_set, test_set = train_test_split(data, test_size=0.25, random_state=0)

svd= SVD(n_factors = 50, random_state=0)
svd.fit(train_set)
predictions = svd.test(test_set)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

### (3) 교차검증과 하이퍼 파라미터 튜닝

In [31]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 로딩
ratings = pd.read_csv('./ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5,5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

svd = SVD(random_state=0)
cross_validate(svd, data, measures=['RMSE','MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8791  0.8621  0.8712  0.8802  0.8798  0.8745  0.0070  
MAE (testset)     0.6717  0.6637  0.6687  0.6760  0.6774  0.6715  0.0050  
Fit time          0.97    0.98    1.00    0.98    0.97    0.98    0.01    
Test time         0.25    0.11    0.11    0.13    0.11    0.14    0.06    


{'test_rmse': array([0.87912528, 0.86211047, 0.87119688, 0.88019938, 0.87978685]),
 'test_mae': array([0.67168259, 0.66367929, 0.66873545, 0.67604441, 0.67736952]),
 'fit_time': (0.967374324798584,
  0.9798262119293213,
  1.0041358470916748,
  0.9773900508880615,
  0.9677796363830566),
 'test_time': (0.2543222904205322,
  0.10766792297363281,
  0.11070489883422852,
  0.12662982940673828,
  0.10688900947570801)}

In [32]:
# 경사하강법을 사용하여 파라미터 튜닝
from surprise.model_selection import GridSearchCV

#최적화할 파라미터 
param_grid = {'n_epochs' : [20, 40, 60], 'n_factors':[50, 100, 200]}
              
# CV를 3개의 폴드로 지정하고 rmse, mse 로 평가수행
gs = GridSearchCV(SVD, param_grid, measures=['rmse','mse'], cv=3)
gs.fit(data)

# 최고의 RMSE 평가 점수 출력
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8781767833719428
{'n_epochs': 20, 'n_factors': 50}


## 3. Surprise 를 사용한 개인화 영화 추천 시스템 구현

#### ratings.csv 데이터를 학습 데이터, 테스트 데이터를 분리하지 않고 전체 데이터를 학습 데이터로 사용하려고 함.
#### Surprise 라이브러리는 trina_test_split() 을 이용해야만 fit()으로 학습할 수 있음.

In [34]:
# 다음의 코드는 오류가 발생함.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
svd = SVD(n_factors=50, random_state=0)
svd.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

#### 데이터세트 전체를 학습 데이터로 사용하려면 DatasetAutoFolds 클래스 이용하여 전체데이터로 지정

In [78]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format = 'user item rating timestamp', sep=',', rating_scale=(0.5, 5.0))
data_folds = DatasetAutoFolds(ratings_file='./ml-latest-small/ratings_noh.csv', reader=reader)

# 전체데이터를 학습데이터로 생성
trainset = data_folds.build_full_trainset()
print(trainset)

#### 학습 수행

In [62]:
svd = SVD(n_epochs=20, n_factors=50, random_state=0)
svd.fit(trainset)

### 특정 사용자의 보지 않은 영화의 평점 예측

#### 특정 사용자 : userId = 9, movieId = 42 지정하고 예측평점 계산

In [63]:
# 영화 정보 데이터프레임 로딩
movies = pd.read_csv('./ml-latest-small/movies.csv')
#movies.head()

# userId=9의 movieId 데이터 추출
movieIds = ratings[ratings['userId'] == 9]['movieId']
if movieIds[movieIds == 42].count() == 0:
    print('사용자 ID 9는 영화 아이디 42의 평점 데이터 없음')

print(movies[movies['movieId']==42])

사용자 ID 9는 영화 아이디 42의 평점 데이터 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


### 영화ID 42의 예측평점 구하기

In [64]:
uid = str(9)
iid = str(42)

pred = svd.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


### 사용자가 평점을 매기지 않은 전체 영화를 추출하는 함수

In [69]:
def get_unseen_surprise(ratings, movies, userId):
    #매개변수의 userId 사용자가 평점을 매긴 영화리스트 추출
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()
    
    # 모든 영화의 movieId 를 리스트로 생성
    total_movies = movies['movieId'].tolist()
    
    # 전체 영화 리스트 - 평점을 매긴 리스트 => 평점을 안 매긴 영화 리스트 
    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화 수: ', len(seen_movies), ', 추천 대상 영화 수: ', len(unseen_movies),
          ', 전체 영화 수 : ', len(total_movies))
    return unseen_movies
unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수:  46 , 추천 대상 영화 수:  9696 , 전체 영화 수 :  9742


### 평점을 안매긴 영화에 대해 예측 평점순으로 정렬후 n개 추천

In [75]:
def recomm_movie_by_surprise(svd, userId, unseen_movies, top_n=10):
    # svd 객체에 unseen_movies를 입력하여 예측평점 계산
    predictions = [svd.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    #[Predictions(userId='9', movieId='1', est=3.5),....]
    
    def sortkey_est(pred):
        return pred.est
    
    # sortkey_est() 변환값의 내림차순으로 정렬 수행
    predictions.sort(key=sortkey_est, reverse=True)
    
    top_predictions = predictions[:top_n]
    
    # top_n으로 추출된 영화의 정보 추출 : iid => movieId
    top_movie_ids = [int(pred.iid) for pred in top_predictions]
    top_movie_rating = [pred.est for pred in top_predictions] # 예측 평점 리스트 추출
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]
    
    return top_movie_preds


In [76]:
# 사용자Id가 9인 사용자의 10개 영화 추천
unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(svd, 9, unseen_movies, top_n=10)

print('#### Top-10 추천 영화 리트스 ####')
for top_movie in top_movie_preds:
    print(top_movie[1], ":" , top_movie[2])

평점 매긴 영화 수:  46 , 추천 대상 영화 수:  9696 , 전체 영화 수 :  9742
#### Top-10 추천 영화 리트스 ####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


In [ ]:
|